### LSTM downstream prediction model

Evaluate an LSTM downstream prediction model on the raw physiological signal embeddings and the static patient data.

## Hyperparameter tuning

In [ ]:
import os
from phase.prediction import *

PATH  = os.path.expanduser("~/phase/")
DPATH = PATH+"downstream_prediction/"
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"

lookback = 60
DEBUG = False

label_type = "desat_bool92_5_nodesat"
curr_feat = "SAO2"
print("\n[Progress] label_type: {}, curr_feat {}".format(label_type, curr_feat))

hosp_data = 0
data_type = "proc[top15]+nonsignal"

# Load train validation data and split it
(X_trval_lst,y_trval) = load_data(DPATH,data_type,label_type,True,
                                  hosp_data,curr_feat,DEBUG=DEBUG)
X_trval_lst = standardize_static(X_trval_lst)

numlayers = [1,2,3]
numnodes  = [100,200,300]
opt_lst   = ["RMSprop", "SGD", "Adam"]
lr_lst    = [0.01,0.001,0.0001]
drop_lst  = [0,0.5]

for numlayer in numlayers:
    for numnode in numnodes:
        for opt in opt_lst[0:1]:
            for lr in lr_lst:
                for drop in drop_lst:
                    hyper_dict = {"numlayer":numlayer,"numnode":numnode,
                                  "opt":opt,"lr":lr,"drop":drop}
                    form_train_model(X_trval_lst,y_trval,hyper_dict,label_type)

## Load min model and train and evaluate

In [ ]:
import os
from phase.prediction import *

PATH  = os.path.expanduser("~/phase/")
DPATH = PATH+"downstream_prediction/"
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"

lookback = 60
DEBUG = False

label_type_eta_currfeat_lst = [("desat_bool92_5_nodesat",0.02,"SAO2"),
                               ("nibpm60",0.1,"NIBPM"), 
                               ("etco235",0.1,"ETCO2")]

for label_type, _, curr_feat in label_type_eta_currfeat_lst:
    print("\n[Progress] label_type: {}, curr_feat {}".format(label_type, curr_feat))

    hosp_data = 0
    data_type = "proc[top15]+nonsignal"

    # Get best model in terms of min validation loss
    TUNEPATH = MODELPATH+"tune_multilstm_0hospdata_desat_bool92_5_nodesatlabel/"
    mod_names = os.listdir(TUNEPATH)
    best_min_val_loss = float("inf")
    for mod_name in mod_names:
        CURRMODPATH = TUNEPATH+mod_name
        model_checkpts = [f for f in os.listdir(CURRMODPATH) if "val_loss" in f]
        min_val_loss = float(sorted(model_checkpts)[1].split("val_loss:")[1].split("_")[0])
        if min_val_loss < best_min_val_loss:
            best_min_val_loss = min_val_loss
            best_mod_name = mod_name

    hyper_dict = {"numlayer" : int(best_mod_name.split("numlayer_")[0]),
                  "numnode"  : int(best_mod_name.split("numnode_")[0].split("_")[-1]),
                  "opt"      : best_mod_name.split("opt_")[0].split("_")[-1],
                  "lr"       : float(best_mod_name.split("lr_")[0].split("_")[-1]),
                  "drop"     : float(best_mod_name.split("drop_")[0].split("_")[-1])}

    # Load train validation data and split it
    (X_trval_lst,y_trval) = load_data(DPATH,data_type,label_type,True,hosp_data,curr_feat,DEBUG=DEBUG)
    X_trval_lst = standardize_static(X_trval_lst)

    # Train model
    MODDIR = form_train_model(X_trval_lst,y_trval,hyper_dict,label_type,is_tune=False,epoch_num=200)

    mod_type = "multilstm_{}".format(label_type)
    RESDIR = '{}{}/'.format(RESULTPATH, mod_type)
    if not os.path.exists(RESDIR): os.makedirs(RESDIR)

    # Load test data
    (X_test1_lst,y_test1) = load_data(DPATH,data_type,label_type,False,hosp_data,curr_feat,DEBUG=DEBUG)
    X_test1_lst = standardize_static(X_test1_lst)
    X_test1_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_test1_lst]
    X_test1_lst = [np.concatenate(X_test1_lst[:-1],2),X_test1_lst[-1]]

    # Evaluate on the test set
    load_model_and_test(RESDIR,MODDIR,X_test1_lst,y_test,data_type,hosp_data)


[Progress] label_type: desat_bool92_5_nodesat, curr_feat SAO2
[DEBUG] Y.shape: (3920564,)
[DEBUG] Starting load_raw_data
[DEBUG] DPATH /homes/gws/hughchen/phase/downstream_prediction//data/desat_bool92_5_nodesat/hospital_0/


W1028 09:03:05.937915 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1028 09:03:05.939880 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1028 09:03:05.943795 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



[PROGRESS] form_model()


W1028 09:03:19.665680 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1028 09:03:19.672633 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1028 09:03:19.677962 140596101756736 deprecation.py:323] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[PROGRESS] Starting train_model()
[PROGRESS] Making MODDIR /homes/gws/hughchen/phase/downstream_prediction/models/multilstm_0hospdata_desat_bool92_5_nodesatlabel_3numlayer_300numnode_RMSpropopt_0.001lr_0.0drop_1000bsize_200epochnum_300000perepochsize/


W1028 09:03:23.311719 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 300000 samples, validate on 392057 samples
Epoch 1/1
300000/300000 [==============================] - 171s 570us/step - loss: 0.0550 - val_loss: 0.0437
Train on 300000 samples, validate on 392057 samples
Epoch 1/1
300000/300000 [==============================] - 166s 554us/step - loss: 0.0449 - val_loss: 0.0410
Train on 300000 samples, validate on 392057 samples
Epoch 1/1
300000/300000 [==============================] - 166s 552us/step - loss: 0.0407 - val_loss: 0.0396
Train on 300000 samples, validate on 392057 samples
Epoch 1/1
299000/300000 [============================>.] - ETA: 0s - loss: 0.0412